# **StomataPy - training**

Contact: hongyuan.zhang@usys.ethz.ch

The file structure:
```
├── StomataPy (the root directory)
    ├── Stomata_detection (data for object detection)
    ├── Stomata_segmentation (data for semantic segmentation)
    ├── train (config files for training)
    ├── Training.ipynb (the file you are running now)
```
There will be one point you need to run `export PYTHONPATH=$PYTHONPATH:/StomataPy/mmsegmentation`, change the `StomataPy` to your `root_dir`

# Data preparation

In [1]:
%cd ..

c:\Users\Charl\OneDrive\Academics\2023 ETH\Projects\PhD thesis\StomataPy


In [1]:
from stomatapy.utils.data_statistics import DataStatistics

selected_jsons = DataStatistics().select_species_folders(pavements_only=True, ensemble_files=True)

c:\Users\Charl\OneDrive\Academics\2023 ETH\Projects\PhD thesis\StomataPy\stomatapy\utils\data_statistics.py:167: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame([result])], axis=0)  # concatenate all results


Total masks: 38837, 17279 (44.49 %) out of which is autolabeled
Category counts: {'pavement cell': 32755, 'stoma': 2976, 'stomatal complex': 2242, 'outer ledge': 864}
slected datasets: ['Vofely2019 - D. californica', 'Vofely2019 - D. californica', 'Vofely2019 - B. setifera', 'Fetter2019 - D. piperita', 'Fetter2019 - R. darienensis', 'Fetter2019 - O. marmellensis', 'Fetter2019 - S. repens', 'Vofely2019 - E. pulcherrima', 'Vofely2019 - C. mirabilis', 'Vofely2019 - C. mirabilis', 'Vofely2019 - C. mirabilis', 'Vofely2019 - C. mirabilis', 'Vofely2019 - C. mirabilis', 'Vofely2019 - C. mirabilis', 'Fetter2019 - P. polyphleba', 'Fetter2019 - S. mukorossi', 'Fetter2019 - C. tepejilote', 'Vofely2019 - D. bulbifera', 'Vofely2019 - D. bulbifera', 'Fetter2019 - K. strobilina', 'Fetter2019 - C. crinita', 'Fetter2019 - C. crinita', 'Fetter2019 - C. crinita', 'Vofely2019 - G. biloba', 'Fetter2019 - E. sasythrasa', 'Fetter2019 - A. impressa', 'Fetter2019 - Q. angustifolia', 'Fetter2019 - S. glomerulifl

In [1]:
from stomatapy.core.isat import UtilsISAT
input_dir = 'Datasets//Ensemble - Copy'

UtilsISAT.select_class(input_dir , action='rename class', source_class='stoma', destination_class='stomatal complex', allow_remove_source_class=True)  # replace non-duplicated stoma with stomatal complex
classes2remove = ['outer ledge', 'pore']
for category in classes2remove:
    UtilsISAT.select_class(input_dir, category=category, action='remove')  # remove catergoreis that are not 'stomatal complex' or 'pavement cell'
UtilsISAT.resize_isat(input_dir=input_dir, new_width=4352, new_height=1844, if_keep_ratio=True)
UtilsISAT.shapely_valid_transform(input_dir)  # transform the polygons to be valid as shapely require

In [2]:
from stomatapy.utils.data4training import Data4Training
input_dir = 'Datasets/Casado-Garcia2020/Processed/H. vulgare/Annotated'
# input_dir = 'Datasets//Ensemble'
Data4Training(input_dir=input_dir, aim='object detection').data4training(if_resize_isat=True)

train size=37, validation size=9
indexing coco dataset annotations...


100%|██████████| 37/37 [00:21<00:00,  1.71it/s]


indexing coco dataset annotations...


100%|██████████| 9/9 [00:04<00:00,  1.83it/s]


# I\/O operations

In [ ]:
!unzip Stomata_segmentation.zip -d Stomata_segmentation

# Check GPUs and CPUs

# Check the config file (optional)

In [17]:
from mmengine import Config
cfg = Config.fromfile('Configs//swin-l_dino.py')
print(cfg.pretty_text)
#cfg.dump('Configs//swin-l_dino_config.py')

# MMDetection

## install mmdetection

In [ ]:
####### code block 1 ########
!mim install "mmdet>=3.0.0rc0"
#!git clone https://github.com/open-mmlab/mmdetection.git
#%cd mmdetection
#!pip install -v -e .
#%cd ..

## visualize dataset

In [ ]:
mim run mmdet browse_dataset train/mmdetection/Swin-l_dino_config.py  --output-dir viz_dataset_mmdet/ 

## training

In [ ]:
mim train mmdet Configs//INSTANCE_Stomata_mask2former_swin-s.py --gpus 4 --launcher pytorch

# Rein

## install mmsegmentation and clone rein

In [25]:
!git clone https://github.com/open-mmlab/mmsegmentation.git
%cd mmsegmentation
!pip install -v -e .
%cd ..
!pip install wandb
!git clone https://github.com/w1oves/Rein.git

fatal: destination path 'mmsegmentation' already exists and is not an empty directory.
/StomataPy/mmsegmentation
Using pip 24.0 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///StomataPy/mmsegmentation
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-sjyeenhy/mmsegmentation.egg-info
  writing manifest file '/tmp/pip-pip-egg-info-sjyeenhy/mmsegmentation.egg-info/SOURCES.txt'
  writing manifest file '/tmp/pip-pip-egg-info-sjyeenhy/mmsegmentation.egg-info/SOURCES.txt'
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mmsegmentation
    Running command python setup.py develop
    running develop
    /usr/local/lib/python3.10/dist-packages/setuptools/command/easy_install.py:156: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
      warnings.warn(


In [26]:
import shutil
from tqdm import tqdm

list2copy = [
    'train/mmsegmentation/__init__.py',
    'train/mmsegmentation/stomata_dataset.py',
    'train/mmsegmentation/cell_dataset.py'
]    

for dir in tqdm(list2copy, total=len(list2copy)):
    shutil.copy2(dir, 'mmsegmentation/mmseg/datasets')

shutil.copy2('train/mmsegmentation/local_visualizer.py', 'mmsegmentation/mmseg/visualization/local_visualizer.py')

shutil.copy2('train/rein/reins_sam_vit.py', 'Rein/rein/models/backbones')
shutil.copy2('train/rein/rein_dinov2_mask2former_config.py', 'Rein/rein_dinov2_mask2former_config.py')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 2179.99it/s]


'Rein/rein_dinov2_mask2former_config.py'

## preview data augmentation (run in terminal)

In [ ]:
cd Rein
export PYTHONPATH=$PYTHONPATH:/StomataPy/mmsegmentation
python ../mmsegmentation/tools/analysis_tools/browse_dataset.py \
    rein_dinov2_mask2former_config.py  \
    --output-dir ../viz_dataset/ 

## get dinov2 weights

In [27]:
%cd Rein
!wget -P checkpoints https://dl.fbaipublicfiles.com/dinov2/dinov2_vitl14/dinov2_vitl14_pretrain.pth
!python tools/convert_models/convert_dinov2.py checkpoints/dinov2_vitl14_pretrain.pth checkpoints/dinov2_converted.pth --height 512 --width 512

/StomataPy/Rein
--2024-05-12 17:58:08--  https://dl.fbaipublicfiles.com/dinov2/dinov2_vitl14/dinov2_vitl14_pretrain.pth
Resolving proxy.ethz.ch (proxy.ethz.ch)... 129.132.202.155
Connecting to proxy.ethz.ch (proxy.ethz.ch)|129.132.202.155|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 1217586395 (1.1G) [binary/octet-stream]
Saving to: ‘checkpoints/dinov2_vitl14_pretrain.pth’

dinov2_vitl14_pretr 100%[===================>]   1.13G   109MB/s    in 11s     

2024-05-12 17:58:19 (104 MB/s) - ‘checkpoints/dinov2_vitl14_pretrain.pth’ saved [1217586395/1217586395]

Load from checkpoints/dinov2_vitl14_pretrain.pth
Convert conv kernel in patch embed layer: torch.Size([1024, 3, 14, 14]) -> torch.Size([1024, 3, 16, 16])
Convert pos embedding: torch.Size([1, 1369, 1024]) -> (-1, 37, 37, 1024) -> torch.Size([1, 1024, 32, 32]) -> torch.Size([1, 1024, 1024])
Save to checkpoints/dinov2_converted.pth


## get config

In [26]:
from mmengine import Config
cfg = Config.fromfile('configs/dinov2/rein_dinov2_mask2former_512x512_bs1x4.py')
print(cfg.pretty_text)
#cfg.dump('rein_dinov2_mask2former_512x512_bs1x4.py')

## training (run in terminal)

In [ ]:
# e98d0d0fc185629c362c4f0f39cc9282d955d217
# --cfg-options efficient_conv_bn_eval="[backbone]"

In [ ]:
export PYTHONPATH=$PYTHONPATH:/StomataPy/mmsegmentation
PORT=12345 CUDA_VISIBLE_DEVICES=0,1,2,3 bash tools/dist_train.sh rein_dinov2_mask2former_config.py 4

In [ ]:
!python tools/generate_full_weights.py \
    --dinov2_segmentor_path checkpoints/dinov2_segmentor_test.pth \
    --backbone checkpoints/dinov2_vitl14_pretrain.pth \
    --rein_head checkpoints/best_mIoU_epoch_155.pth

In [ ]:
import torch
import torch.nn as nn
import rein
from mmengine.config import Config
from mmseg.apis.inference import inference_model,show_result_pyplot,init_model
import matplotlib.pyplot as plt
cfg=Config.fromfile('configs/dinov2/rein_dinov2_mask2former_test.py')
network:nn.Module=init_model(cfg,'checkpoints/dinov2_segmentor_test.pth','cuda:0')
network.cfg=cfg
#network.cfg.test_pipeline=network.cfg.bdd_test_pipeline
torch.set_grad_enabled(False)

In [ ]:
im_path='VegAnn_326.png'
result=inference_model(network,im_path)
vis_img=show_result_pyplot(network,im_path,result)
plt.imshow(vis_img)
plt.axis('off')